Лабораторная работа №1. Извлечение ключевых слов из текста.

В файле README.md лежат некоторые пояснения, ссылка на исходные ключевые слова, а также возможный ответ на 5 вопрос

Ниже представлены используемые библиотеки:

In [1]:
import pprint
import pandas as pd

from nlp_rake import Rake
from summa import keywords
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

Зададим функции для предобработки текста:

In [2]:
def get_preload_text():
    """
    Запись предобработанного текста
    """
    for i in range(1, 5):
        with open(f"./packs/N{i}/text.txt", encoding='utf-8') as f:
            res_text = preload(f.read())
        with open(f"./packs/N{i}/res_text.txt", 'w', encoding='utf-8') as f:
            f.write(res_text)


def preload(text):
    """
    Предобработка текста (лемматизация)
    """

    punctuation_signs = list("?:!.,;")

    res = text.replace("\r", " ")
    res = text.replace("\n", " ")
    res = res.replace("    ", " ")
    res = res.replace('"', '')
    res = res.lower()
    for punct_sign in punctuation_signs:
        res = res.replace(punct_sign, '')
    sentences = [word_tokenize(i, language="russian") for i
                 in sent_tokenize(res, language="russian")]

    # Приведем к нормальной форме
    morph = MorphAnalyzer()
    sentences = [[morph.parse(i)[0].normal_form for i in sen]
                 for sen in sentences]

    # Удалим стоп слова
    russian_stopwords = stopwords.words("russian")
    res_sentences = [[word for word in sen if word not in russian_stopwords]
                     for sen in sentences]

    # Соберем в кучу
    res_text = ' '.join(sentences[0])
    return res_text

Ниже определены функции, необходимые при методе Tf-Idf:

In [3]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)


def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """
    Определение ревелантных ключей
    :param feature_names:
    :param sorted_items:
    :param topn:            количество
    :return:                словарь с ключами и значениями - частотой
    """
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    results = {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]] = score_vals[idx]

    return results

Определим класс, с помощью которого мы будем выделять ключевые слова из текстов:

In [4]:
class Extraction:
    """
    Класс, описывающий работу поиска ключевых слов из текста
    """

    def __init__(self):
        # Массивы ключевых слов для текстов
        self.keywords = []              # Исходыне
        self.manual_keywords = []       # Составленные вручную
        self.textrank_keywords = []     # С методом TextRank
        self.tfidf_keywords = []        # С методом TfIdf
        self.rake_keywords = []         # С методом RAKE

    def get_keywords(self):
        """
        Вычленение ключевых слов из файла
        """
        for i in range(1, 5):
            df = pd.read_csv(f"./packs/N{i}/keys.csv", sep='|')
            set_in = set()
            for row in df["lemm"].to_list():
                set_in.add(row)
            self.keywords.append(set_in)

    def manual_markup(self):
        """
        Создание объединенных множеств ключевых слов
        """
        for i in range(1, 5):
            df = pd.read_csv(f'./packs/keywords{i}.csv', sep='|')
            set_in = set()
            for row in df['tags'].to_list():
                set_in.add(row)
            res_words = set_in.intersection(self.keywords[i-1])
            self.manual_keywords.append(res_words)

    def textrank_extraction(self):
        """
        TextRank способ извлечения
        """
        for i in range(1, 5):
            with open(f"./packs/N{i}/res_text.txt", encoding='utf-8') as f:
                res_text = f.read()
            self.textrank_keywords.append(
                set(keywords.keywords(res_text, language='russian', split=True,
                                      additional_stopwords=stopwords.words("russian"))))

    def rake_extraction(self):
        """
        RAKE способ извлечения
        """
        for i in range(1, 5):
            with open(f"./packs/N{i}/res_text.txt", encoding='utf-8') as f:
                res_text = f.read()
                rake = Rake(language_code='ru', max_words=1)
                keywords = rake.apply(res_text)

                set_in = set()
                for i in keywords:
                    set_in.add(i[0])
                self.rake_keywords.append(set_in)

    def tfidf_extraction(self):
        """
        TfIdf способ извлечения
        """
        pass
        texts = []
        for i in range(1, 5):
            with open(f"./packs/N{i}/res_text.txt", encoding='utf-8') as f:
                texts.append(f.read())

        cv = CountVectorizer(max_df=0.85, stop_words=stopwords.words("russian"), max_features=150)
        word_count_vector = cv.fit_transform(texts)
        tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
        tfidf_transformer.fit(word_count_vector)

        feature_names = cv.get_feature_names_out()

        for doc in texts:
            tf_idf_vector = tfidf_transformer.transform(cv.transform([doc]))
            sorted_items = sort_coo(tf_idf_vector.tocoo())

            res = set()
            for key in extract_topn_from_vector(feature_names, sorted_items, 30):
                res.add(key)
            self.tfidf_keywords.append(res)

    def pprint(self):
        """
        Проверка введённых/извлеченных ключевых слов
        """
        pp = pprint.PrettyPrinter(compact=True)
        print('Keywords')
        pp.pprint(self.keywords)
        print('---------')
        print('Manual Keywords')
        pp.pprint(self.manual_keywords)
        print('---------')
        print('TextRank Keywords')
        pp.pprint(self.textrank_keywords)
        print('---------')
        print('TfIdf Keywords')
        pp.pprint(self.tfidf_keywords)
        print('---------')
        print('RAKE Keywords')
        pp.pprint(self.rake_keywords)

    def pprint_common(self):
        pp = pprint.PrettyPrinter(compact=True)
        print('---------')
        print('TextRank+TfIdf Keywords')
        common = [i.intersection(j) for i, j in zip(self.textrank_keywords,
                                                   self.tfidf_keywords)]
        pp.pprint(common)

        print('---------')
        print('TextRank+RAKE Keywords')
        common = [i.intersection(j) for i, j in zip(self.textrank_keywords,
                                                   self.rake_keywords)]
        pp.pprint(common)
        print('---------')
        print('TfIdf+RAKE Keywords')
        common = [i.intersection(j) for i, j in zip(self.tfidf_keywords,
                                                   self.rake_keywords)]
        pp.pprint(common)

Предобработаем тексты и найдём ключевые слова в тексте:

In [5]:
get_preload_text()

# Определим класс для работы
a = Extraction()

# Загрузим ключевые слова, которые были помечены, а так же сравним с нашими собственными
a.get_keywords()
a.manual_markup()

# Используем методы выделения ключевых слов
a.textrank_extraction()
a.rake_extraction()
a.tfidf_extraction()

# А теперь выпишем их
a.pprint()


Keywords
[{'10 июля', '100 спутников', '20 отраслях', '200 действующих',
  '2020-2025 годах', '2025 году', '30 искусственных', '307 запусков', '39 в',
  'аппарат', 'год', 'китай', 'кнр', 'космический', 'марс', 'национальный', 'нс',
  'пекин', 'прорыв', 'страна', 'тасс', 'технология', 'ци', 'юц'},
 {'1', '1 миллиард', '1 трлн', '1,2 трлн', '10 годами', '111.4 указанного',
  '12 суперкластеров', '14 млрд', '15-процентного', '2017 г', '2025 году',
  '2030 году', '22 апреля 2019 года', '27 инновационно-территориальных',
  '30 протокола', '4', '43 промышленных', '48 части 1 статьи 93 44-фз',
  '600 организаций', '85 новых', 'аеа', 'ар', 'ас', 'баз', 'биокад',
  'боеприпас', 'внешэкономбанк', 'внии', 'военнопромышленный', 'военный',
  'возможность', 'восход', 'вп', 'вузы', 'высокотехнологичный', 'гоз',
  'госзаказ', 'госкорпорация', 'государственный', 'государство', 'гпб',
  'гражданский', 'группа', 'да', 'данный', 'двойной', 'дг', 'диверсификация',
  'до', 'до 30%', 'до 50%', 'договор', 'до